In [6]:
import csv
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import pickle
import re
import numpy as np
import glob
import os
os.getcwd()

'/home/matthew/anaconda3/envs/insight/staycationLA/notebooks'

In [2]:
os.chdir('/home/matthew/anaconda3/envs/insight/staycationLA/notebooks')
os.getcwd()

'/home/matthew/anaconda3/envs/insight/staycationLA/notebooks'

In [79]:
# function to check if the button is on the page, to avoid miss-click problem
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [80]:
def scrape(driver, user):

    if (check_exists_by_xpath("//a[contains(@class, 'ui_tab social-profile-ProfileTab__ui_tab--3f5yN') and contains(@data-tab-name, 'Travel map')]")):
        # to open travel map tab
        driver.find_element_by_xpath("//a[contains(@class, 'ui_tab social-profile-ProfileTab__ui_tab--3f5yN') and contains(@data-tab-name, 'Travel map')]").click()
        time.sleep(1)
        
        driver.switch_to.window(driver.window_handles[1])
        mapURL = (driver.current_url)
        map_source = requests.get(mapURL)
        m = re.findall('''"flags":\[([a-z, "]*?)\],"name":"(.*?)"''', map_source.text)
        n = [(tag, place, user) for (tag, place) in m]
        df = pd.DataFrame(n, columns=["tag", "place", "user"])
        df.to_csv(r'../data/raw/user_LA_' + user + '.csv', index = False, header=True)
        #df.to_csv(r'/raw/user_LA_' + user + '.csv', index = False, header=True)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    driver.close()

In [81]:
#load list of usernames from NB04 LA attraction reviews combine
with open('../data/processed/userlist_final.pkl', 'rb') as f:
    usernames2 = pickle.load(f)
len(usernames2)

80277

In [82]:

# converting list to array 
usernames3 = np.array(usernames2) 
len(usernames3)

80277

In [83]:
#remove duplicates
names=[ x for x in usernames3 if " " not in x ]
len(names)

80250

In [84]:
os.chdir("/home/matthew/anaconda3/envs/insight/staycationLA/data/raw")
extension = 'csv'
all_users = [i for i in glob.glob('user_LA_*.{}'.format(extension))]
#print(all_users)

user_got = []

for string in all_users:
    string1 = str(string)       
    new_string = string1.replace("user_LA_", "")
    new_string2 = new_string.replace(".csv", "")
    user_got.append(new_string2)

user_got = list(dict.fromkeys(user_got)) #remove duplicates
len(user_got)

86161

In [85]:
name_list = np.setdiff1d(names,user_got)
# yields the elements in `list_2` that are NOT in `list_1`
len(name_list)

1860

In [86]:
name_list2=name_list.tolist()

In [87]:
#split list in half to run two scrapers at same time
names1 = name_list2[:len(name_list2)//2]
names2 = name_list2[len(name_list2)//2:]

In [62]:
len(names1)

1466

In [3]:
#print(names1)

In [27]:
len(names2)

20855

In [4]:
#print(names2)

In [90]:
#reset directory bc I changed it in the glob earlier
os.chdir('/home/matthew/anaconda3/envs/insight/staycationLA/notebooks')
os.getcwd()

'/home/matthew/anaconda3/envs/insight/staycationLA/notebooks'

In [16]:
for user in names1:
    driver = webdriver.Chrome("/home/matthew/Downloads/chromedriver")
    # insert the tripadvisor's website of one attraction 
    driver.get("https://www.tripadvisor.com/Profile/" + user)
    time.sleep(5)
    scrape(driver, user)

KeyboardInterrupt: 

In [29]:
for user in names2:
    driver = webdriver.Chrome("/home/matthew/Downloads/chromedriver")
    # insert the tripadvisor's website of one attraction 
    driver.get("https://www.tripadvisor.com/Profile/" + user)
    time.sleep(5)
    scrape(driver, user)

KeyboardInterrupt: 

In [ ]:
os.chdir("/home/matthew/anaconda3/envs/insight/staycationLA/data/raw")

#glob all the user travel history files into a list
extension = 'csv'
all_filenames = [i for i in glob.glob('user_LA_*.{}'.format(extension))]
#print(all_filenames)
    
#combine all files in the list into df, drop US and Canada history, and save it as csv
li = []

for filename in all_filenames:
    df = pd.read_csv(filename, index_col=None)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame2=frame.dropna(axis=0, how='any') 
frame3=frame2[~frame2.place.str.contains(", USA")] #remove all domestic rows
frame4=frame3[~frame2.place.str.contains(", Canada")] 
#export to csv
frame4.to_csv( "../processed/LA_combined_user_travel_hist.csv", index=False, encoding='utf-8-sig')

In [8]:
len(frame)

6942052